## Вывод полуаналитических зависимостей для метрики $d_{\text{max}}$ (изохронное отклонение)

Вид линий уровня для графика $r_{min}(\sigma_{pos}, \sigma_{vel})$ поддатлкивает на мысль о том, что зависимость должна быть такой:
$$
r_{min} = \sqrt[n]{\alpha_1\cdot\sigma_{pos}^n+\alpha_2\cdot\sigma_{vel}^n}
$$

Для нахождения коэффициентов $\alpha = (\alpha_1, \alpha_2)$ воспользуемся формулой функционала для линейной регресии:
$$
J_2 = \dfrac{1}{N}\sum_{i=1}^n(r_{min}^n-\alpha_1\cdot\sigma_{pos}^n-\alpha_2\cdot\sigma_{vel}^n)^2 \longrightarrow \min_{\alpha_1, \alpha_2}
$$

Для такого функционала есть ответ. Если переписать его ядро в матричном виде, то по формуле Маркова-Гаусса:
$$
\boxed{\alpha^*=(A^TA)^{-1}A^Ty} = \alpha^*(n)
$$

In [1]:
from IPython.display import display, Math

import numpy as np

from utils.libration_sense import get_xf, km2du, kmS2vu, du2km, get_maxdeviation_wo_integrate
from utils.formula_creators import n_finder, n_finder_performed, alpha_finder_of_n

In [2]:
orbit_type = "L1"
orbit_number = 92

In [3]:
# ищем все параметры для формулы
xfinal = get_xf(orbit_type, orbit_number)
n, A_normed, y_du = n_finder(orbit_type, orbit_number, xfinal, grid_density=20, seed=42, reuse_noise=True, amount_of_points=5_000)
alpha, deviation_max = alpha_finder_of_n(A_normed, y_du, n)

print('Степень n = ', n, sep='')
print("alpha1, alpha2 = ", alpha)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/i.zabara/VSCodeProjects/halo-sensitivity/utils/../data/input/qualified_ic_halo/HOPhaseVectorsEarthMoonL1_qualified.csv'

In [ ]:
# ищем все параметры для формулы с помощью оптимизированной процедуры
xfinal = get_xf(orbit_type, orbit_number)
n_perf, A_normed, y_du = n_finder_performed(orbit_type, orbit_number, xfinal, grid_density=20, seed=42, reuse_noise=True, amount_of_points=5_000)
alpha, deviation_max = alpha_finder_of_n(A_normed, y_du, n_perf)

print('Степень n = ', n_perf, sep='')
print("alpha1, alpha2 = ", alpha)
print("deviation max = ", du2km(deviation_max))

Степень n = 2.3454377366389974
alpha1, alpha2 =  [0.61144315 0.34603848]
deviation max =  14375.505142388189


In [ ]:
n = n_perf
def deviation_analytic(n, alpha, std_pos_normed, std_vel_normed):
    return np.power(alpha[0]*np.power(std_pos_normed, n) + alpha[1]*np.power(std_vel_normed, n), 1/n)


formula = f"d = \\sqrt[{np.around(n, 4)}]{{{np.around(alpha[0], 2)}\\cdot \\sigma_{{pos}}^{{{np.around(n, 4)}}} + {np.around(alpha[1], 2)}\\cdot \\sigma_{{vel}}^{{{np.around(n, 4)}}}}}\\cdot{np.around(du2km(deviation_max), 3)} \\ [km]"

std_pos, std_vel = km2du(2.4), kmS2vu(0.01e-3)
std_pos_max, std_vel_max = km2du(8), kmS2vu(0.05e-3)

print(f"Пробная точка: [sigma_pos = {4} км, sigma_vel = {0.03} м/с]")
print(f"Крайние значения данных: [sigma_pos_max = {8} км, sigma_vel_max = {0.05} м/с]")
print('Точное значение: ', du2km(get_maxdeviation_wo_integrate(orbit_type, orbit_number, xfinal, std_pos, std_vel, amount_of_points=5000, seed = 42)))
print('Значение, вычисленное по аппроксимирующей формуле: ', deviation_analytic(n, alpha, std_pos/std_pos_max, std_vel/std_vel_max) * du2km(deviation_max))

In [ ]:
formula = f"d = \\sqrt[{np.around(n, 4)}]{{{np.around(alpha[0], 2)}\\cdot \\sigma_{{pos}}^{{{np.around(2, 4)}}} + {np.around(alpha[1], 2)}\\cdot \\sigma_{{vel}}^{{{np.around(n, 4)}}}}}\\cdot{np.around(du2km(deviation_max), 3)} \\ [km]"
display(Math(formula))